In [1]:
import os
import sys
import urllib
import numpy as np
import pandas as pd
#import geopandas as gpd
# import matplotlib.pyplot as plt
import psycopg2
from sqlalchemy import create_engine

sys.path.append('../')
from envir import config
#%matplotlib inline

In [2]:
engine = create_engine('postgresql+psycopg2:///yelp_abcs')
con = engine.connect()

In [3]:
! curl https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD > nycinspect.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  161M    0  161M    0     0  7498k      0 --:--:--  0:00:22 --:--:-- 7161k


In [4]:
nyc_inspect = pd.read_csv('nycinspect.csv')

os.system('mv nycinspect.csv '+ config.shared)

nyc_inspect.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,50035073,BIRCH COFFEE,Manhattan,432,3RD AVE,10016.0,9178416564,Café/Coffee/Tea,10/07/2019,Violations were cited in the following area(s).,...,11/10/2019,Cycle Inspection / Initial Inspection,40.743295,-73.979903,106.0,2.0,7200.0,1083072.0,1.008868e+09,MN20
1,50070300,POKE JUNKE,Manhattan,38,W 38TH ST,10018.0,9178822898,Chinese/Japanese,01/23/2019,Violations were cited in the following area(s).,...,11/10/2019,Cycle Inspection / Initial Inspection,40.751505,-73.984058,105.0,4.0,8400.0,1015979.0,1.008390e+09,MN17
2,41623838,HAPPY DRAGON,Bronx,12791281,WALTON AVENUE,NaN,7185886700,Chinese,03/08/2017,Violations were cited in the following area(s).,...,11/10/2019,Cycle Inspection / Initial Inspection,0.000000,0.000000,NaN,NaN,NaN,NaN,2.000000e+00,NaN
3,40371988,NORTH SHORE DINER,Queens,19652,NORTHERN BLVD,11358.0,7182258000,American,06/15/2017,Violations were cited in the following area(s).,...,11/10/2019,Cycle Inspection / Initial Inspection,40.757847,-73.784564,411.0,19.0,145101.0,4124565.0,4.055200e+09,QN48
4,40529203,DRAGON CITY,Bronx,2102,BARTOW AVE,10475.0,7186718787,Chinese,05/18/2016,Violations were cited in the following area(s).,...,11/10/2019,Cycle Inspection / Initial Inspection,40.869400,-73.825623,210.0,12.0,46201.0,2128425.0,2.051411e+09,BX13


In [5]:
# move to db
nyc_inspect.to_sql('nyc_inspections', engine, schema='stag', index=False)

In [6]:
nyc_inspect.shape

(395694, 26)

In [7]:
# Number of unique restaurants

nyc_inspect['CAMIS'].nunique()

27031

In [8]:
nyc_inspect.columns

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE', 'Latitude', 'Longitude',
       'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL',
       'NTA'],
      dtype='object')

In [9]:
nyc_inspect['INSPECTION DATE'].min()

'01/01/1900'

The date 01/01/1900 refers to restaurants that haven't been inspected yet.

In [10]:
nyc_inspect['INSPECTION DATE'].max()

'12/31/2018'

In [11]:
nyc_active = nyc_inspect[nyc_inspect['INSPECTION DATE'] > '01/01/1900'].copy()

In [12]:
nyc_active.shape

(394096, 26)

Removed about 2,000 rows from dataset. Now contains only restaurants that have been inspected.

In [13]:
nyc_active['INSPECTION DATE'].min()

'01/02/2016'

In [14]:
nyc_active['VIOLATION DESCRIPTION'].unique

<bound method Series.unique of 0         Evidence of mice or live mice present in facil...
1         Single service item reused, improperly stored,...
2         Plumbing not properly installed or maintained;...
3         Food contact surface not properly washed, rins...
4         Plumbing not properly installed or maintained;...
5                Hot food item not held at or above 140º F.
6         Appropriately scaled metal stem-type thermomet...
7         Food contact surface not properly washed, rins...
8         Food not protected from potential source of co...
9         Facility not vermin proof. Harborage or condit...
10        Pesticide use not in accordance with label or ...
11        Facility not vermin proof. Harborage or condit...
12        Facility not vermin proof. Harborage or condit...
13        Sanitized equipment or utensil, including in-u...
14        Food not protected from potential source of co...
15               Hot food item not held at or above 140º F.
16       

In [15]:
nyc_active['VIOLATION CODE'].nunique()

106

In [16]:
nyc_active['VIOLATION DESCRIPTION'].nunique()

91

In [17]:
nyc_active['VIOLATION DESCRIPTION']

0         Evidence of mice or live mice present in facil...
1         Single service item reused, improperly stored,...
2         Plumbing not properly installed or maintained;...
3         Food contact surface not properly washed, rins...
4         Plumbing not properly installed or maintained;...
5                Hot food item not held at or above 140º F.
6         Appropriately scaled metal stem-type thermomet...
7         Food contact surface not properly washed, rins...
8         Food not protected from potential source of co...
9         Facility not vermin proof. Harborage or condit...
10        Pesticide use not in accordance with label or ...
11        Facility not vermin proof. Harborage or condit...
12        Facility not vermin proof. Harborage or condit...
13        Sanitized equipment or utensil, including in-u...
14        Food not protected from potential source of co...
15               Hot food item not held at or above 140º F.
16        Sanitized equipment or utensil

In [18]:
nyc_violations = pd.DataFrame({'violation_code':nyc_active['VIOLATION CODE'], 
                               'violation_description':nyc_active['VIOLATION DESCRIPTION']})

In [19]:
nyc_violations.head()

,violation_code,violation_description
0,04L,Evidence of mice or live mice present in facil...
1,10I,"Single service item reused, improperly stored,..."
2,10B,Plumbing not properly installed or maintained;...
3,06D,"Food contact surface not properly washed, rins..."
4,10B,Plumbing not properly installed or maintained;...


In [20]:
nyc_violations.to_csv(config.shared + '/nyc_violations.csv')

In [21]:
nyc_violations.to_sql('nyc_violations', engine, schema = 'stag', index = False)